In [190]:
import os
import csv
import subprocess
import re
import random
import numpy as np
from numpy import linalg as LA
import pandas as pd

In [93]:
def read_characters():
    tuples = []
    characters = {''}
    with open('will_play_text.csv') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
          charactor = row[4]
          characters.add(charactor)
          line = row[5]
          line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
          line_tokens = [token.lower() for token in line_tokens]
          tuples.append((charactor, line_tokens))
    return tuples,characters

In [95]:
ctuples,charac = read_characters()

In [162]:
charac.discard('')
l_cha = list(charac)

In [164]:
len(l_cha)

934

In [4]:
def read_in_shakespeare():
  '''Reads in the Shakespeare dataset processes it into a list of tuples.
     Also reads in the vocab and play name lists from files.

  Each tuple consists of
  tuple[0]: The name of the play
  tuple[1] A line from the play as a list of tokenized words.

  Returns:
    tuples: A list of tuples in the above format.
    document_names: A list of the plays present in the corpus.
    vocab: A list of all tokens in the vocabulary.
  '''

  tuples = []

  with open('will_play_text.csv') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
      play_name = row[1]
      line = row[5]
      line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
      line_tokens = [token.lower() for token in line_tokens]

      tuples.append((play_name, line_tokens))

  with open('vocab.txt') as f:
    vocab =  [line.strip() for line in f]

  with open('play_names.txt') as f:
    document_names =  [line.strip() for line in f]

  return tuples, document_names, vocab

def get_row_vector(matrix, row_id):
  return matrix[row_id, :]

def get_column_vector(matrix, col_id):
  return matrix[:, col_id]

def create_term_document_matrix(line_tuples, document_names, vocab):
  '''Returns a numpy array containing the term document matrix for the input lines.

  Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    document_names: A list of the document names
    vocab: A list of the tokens in the vocabulary
    
  Let m = len(vocab) and n = len(document_names).

  Returns:
    td_matrix: A mxn numpy array where the number of rows is the number of words
        and each column corresponds to a document. A_ij contains the
        frequency with which word i occurs in document j.
  '''

  vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
  docname_to_id = dict(zip(document_names, range(0, len(document_names))))
  tdm = np.zeros(shape=(len(vocab),len(document_names)))
  for line in line_tuples:
    doc = line[0]
    y_axis = docname_to_id.get(doc)
    for i in range(0,len(line[1])):
      x_axis = vocab_to_id.get(line[1][i])
      tdm[x_axis,y_axis] += 1

  # YOUR CODE HERE
  return tdm

In [154]:
def create_term_character_matrix(line_tuples, characters, vocab):
    ''' Returns a numpy array containing the term character matrix for the input lines.
    '''
    vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
    cha_to_id = dict(zip(characters,range(0,len(characters))))
    tdm = np.zeros(shape=(len(characters),len(vocab)))
    for line in line_tuples:
        cha = line[0]
        x_axis = cha_to_id.get(cha)
        if x_axis is None:
            continue
        for i in range(0,len(line[1])):
          y_axis = vocab_to_id.get(line[1][i])
          tdm[x_axis,y_axis] += 1
            
    return tdm

In [155]:
print('Computing term character matrix...')
tch_matrix = create_term_character_matrix(ctuples,charac,vocab)

Computing term character matrix...


In [156]:
tch_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [157]:
sum_for_row = np.sum(tch_matrix,axis=1)

In [158]:
mask = []
for i in range(len(charac)):
    if sum_for_row[i]< 1100:
        mask.append(i)

In [159]:
len(mask)

688

In [160]:
masked_tc = np.delete(tc_matrix,mask,0)

In [189]:
random_idx = random.randint(0, len(document_names)-1)
similarity_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]
for sim_fn in similarity_fns:
    print('\nThe top most similar character to "%s" using %s are:' % (l_cha[random_idx], sim_fn.__qualname__))
    ranks = rank_plays(random_idx, masked_tc, sim_fn)
    for idx in range(0, 1):
      doc_id = ranks[idx]
      print('%d: %s' % (idx+1, l_cha[doc_id]))


The top most similar character to "Second Clown" using compute_cosine_similarity are:


/Users/weihaoran/env/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


1: Legate

The top most similar character to "Second Clown" using compute_jaccard_similarity are:


/Users/weihaoran/env/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


1: Legate

The top most similar character to "Second Clown" using compute_dice_similarity are:


/Users/weihaoran/env/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


1: Legate


In [5]:
tuples, document_names, vocab = read_in_shakespeare()

print('Computing term document matrix...')
td_matrix = create_term_document_matrix(tuples, document_names, vocab)

Computing term document matrix...


In [6]:
len(tuples)

111396

In [7]:
td_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 5., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [10]:
def create_term_context_matrix(line_tuples, vocab, context_window_size=1):
  '''Returns a numpy array containing the term context matrix for the input lines.

  Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    vocab: A list of the tokens in the vocabulary

  Let n = len(vocab).

  Returns:
    tc_matrix: A nxn numpy array where A_ij contains the frequency with which
        word j was found within context_window_size to the left or right of
        word i in any sentence in the tuples.
  '''

  vocab_to_id = dict(zip(vocab, range(0, len(vocab))))

  # YOUR CODE HERE
  tcm = np.zeros(shape=(len(vocab),len(vocab)))
  for line in line_tuples:
    sent= line[1]
    for i in range(0,len(line[1])):
      left = i - context_window_size
      right = i + context_window_size
      left = 0 if left < 0 else left
      right = len(sent) - 1 if right >= len(sent) else right
      for j in range(left, right+1):
        x_axis = vocab_to_id.get(sent[i],None)
        y_axis = vocab_to_id.get(sent[j],None)
        tcm[x_axis][y_axis] += 1

  return tcm

In [11]:
print('Computing term context matrix...')
tc_matrix = create_term_context_matrix(tuples, vocab, context_window_size=2)

Computing term context matrix...


In [12]:
tc_matrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 47., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., 23.]])

In [19]:
def create_PPMI_matrix(term_context_matrix):
  '''Given a term context matrix, output a PPMI matrix.
    
  Hint: Use numpy matrix and vector operations to speed up implementation.
  
  Input:
    term_context_matrix: A nxn numpy array, where n is
        the numer of tokens in the vocab.
  
  Returns: A nxn numpy matrix, where A_ij is equal to the
     point-wise mutual information between the ith word
     and the jth word in the term_context_matrix.
  '''       
  
  # YOUR CODE HERE
  para = term_context_matrix.shape
  tcm = term_context_matrix.copy() + 1
  n = para[0]
  totals1 = np.sum(term_context_matrix)+ n * n
  totals = np.tile(totals1,(n,para[1]))
  ppmi = np.zeros(para)
  rows1 = np.sum(term_context_matrix,axis=1) + n
  rows = np.tile(rows1,(n,1)).T
  cols1 = np.sum(term_context_matrix,axis=0) + n
  cols = np.tile(cols1,(n,1))
  dom = np.multiply(rows,cols)
  num = np.multiply(totals,tcm) 
  ppmi_final = np.divide(num,dom)
#   for i in range(0,para[0]):
#     for j in range (0,para[1]):
#       ppmi[i][j] = ((term_context_matrix[i][j] + 1) * totals) / (rows[i] * cols[j])
  ppmi_final = np.log2(ppmi_final)
  ppmi_final = np.maximum(ppmi_final, 0)

  return ppmi_final

In [20]:
print('Computing PPMI matrix...')
PPMI_matrix = create_PPMI_matrix(tc_matrix)

Computing PPMI matrix...


In [21]:
PPMI_matrix

array([[1.00954472e+00, 7.12152533e-03, 0.00000000e+00, ...,
        9.41708477e-03, 9.41708477e-03, 2.79540916e-03],
       [7.12152533e-03, 3.46412995e+00, 0.00000000e+00, ...,
        6.99388708e-03, 6.99388708e-03, 3.72211475e-04],
       [0.00000000e+00, 0.00000000e+00, 5.57174896e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.41708477e-03, 6.99388708e-03, 0.00000000e+00, ...,
        1.00928945e+00, 9.28944653e-03, 2.66777092e-03],
       [9.41708477e-03, 6.99388708e-03, 0.00000000e+00, ...,
        9.28944653e-03, 1.00928945e+00, 2.66777092e-03],
       [2.79540916e-03, 3.72211475e-04, 0.00000000e+00, ...,
        2.66777092e-03, 2.66777092e-03, 4.58100860e+00]])

In [22]:
PPMI_matrix

array([[1.00954472e+00, 7.12152533e-03, 0.00000000e+00, ...,
        9.41708477e-03, 9.41708477e-03, 2.79540916e-03],
       [7.12152533e-03, 3.46412995e+00, 0.00000000e+00, ...,
        6.99388708e-03, 6.99388708e-03, 3.72211475e-04],
       [0.00000000e+00, 0.00000000e+00, 5.57174896e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.41708477e-03, 6.99388708e-03, 0.00000000e+00, ...,
        1.00928945e+00, 9.28944653e-03, 2.66777092e-03],
       [9.41708477e-03, 6.99388708e-03, 0.00000000e+00, ...,
        9.28944653e-03, 1.00928945e+00, 2.66777092e-03],
       [2.79540916e-03, 3.72211475e-04, 0.00000000e+00, ...,
        2.66777092e-03, 2.66777092e-03, 4.58100860e+00]])

In [39]:
def create_tf_idf_matrix(term_document_matrix):
  '''Given the term document matrix, output a tf-idf weighted version.
  
  Hint: Use numpy matrix and vector operations to speed up implementation.

  Input:
    term_document_matrix: Numpy array where each column represents a document 
    and each row, the frequency of a word in that document.

  Returns:
    A numpy array with the same dimension as term_document_matrix, where
    A_ij is weighted by the inverse document frequency of document h.
  '''

  # YOUR CODE HERE
  tf_idf_matrix = np.zeros(term_document_matrix.shape)
  df_raw = term_document_matrix.copy()
  tf_raw = term_document_matrix.copy()
  df_raw[df_raw > 0] = 1
  df = np.sum(df_raw,axis=1)
  docs = term_document_matrix.shape[1]
  idf = np.log(docs/df[df>0])

  tf_raw[tf_raw>0] = np.log10(tf_raw[tf_raw>0])+1

  tf = tf_raw

  for row in range(idf.shape[0]):
      tf_idf_matrix[row] = tf[row] * idf[row]

  return tf_idf_matrix


In [40]:
print('Computing tf-idf matrix...')
tf_idf_matrix = create_tf_idf_matrix(td_matrix)

Computing tf-idf matrix...


In [41]:
tf_idf_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 2.19722458, 0.        ,
        0.        ],
       [0.        , 1.17763627, 0.        , ..., 0.69314718, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.28093385, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [32]:
def compute_cosine_similarity(vector1, vector2):
  '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''
  cs = np.dot(vector1,vector2)/(LA.norm(vector1)*LA.norm(vector2))
  # YOUR CODE HERE
  return cs

def compute_jaccard_similarity(vector1, vector2):
  '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''
  
  # YOUR CODE HERE
  num = np.minimum(vector1,vector2)
  dom = np.maximum(vector1,vector2)
  js = np.sum(num)/np.sum(dom)

  return js



In [34]:
def compute_dice_similarity(vector1, vector2):
  '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''

  # YOUR CODE HERE
  upper = np.minimum(vector1,vector2)
  upper_sum = np.sum(upper) * 2
  dom = np.sum(vector1 + vector2)
  return upper_sum/dom


In [35]:
def rank_plays(target_play_index, term_document_matrix, similarity_fn):
  ''' Ranks the similarity of all of the plays to the target play.

  Inputs:
    target_play_index: The integer index of the play we want to compare all others against.
    term_document_matrix: The term-document matrix as a mxn numpy array.
    similarity_fn: Function that should be used to compared vectors for two
      documents. Either compute_dice_similarity, compute_jaccard_similarity, or
      compute_cosine_similarity.

  Returns:
    A length-n list of integer indices corresponding to play names,
    ordered by decreasing similarity to the play indexed by target_play_index
  '''
  
  # YOUR CODE HERE
  nums = term_document_matrix.shape[1]
  target = term_document_matrix.T[target_play_index,:]
  docs_ranking = {}
  result = []
  td_in_cols = term_document_matrix.T
  for i in range(nums):
          if i!= target_play_index:
              similarity_doc = similarity_fn(td_in_cols[i,:], target)
              docs_ranking[i] = similarity_doc
  sort_ranking = sorted(docs_ranking.items(),key=lambda item: item[1],reverse=True)
  for k,v in sort_ranking:
      result.append(k)
  return result



In [36]:
def rank_words(target_word_index, matrix, similarity_fn):
  ''' Ranks the similarity of all of the words to the target word.

  Inputs:
    target_word_index: The index of the word we want to compare all others against.
    matrix: Numpy matrix where the ith row represents a vector embedding of the ith word.
    similarity_fn: Function that should be used to compared vectors for two word
      ebeddings. Either compute_dice_similarity, compute_jaccard_similarity, or
      compute_cosine_similarity.

  Returns:
    A length-n list of integer word indices, ordered by decreasing similarity to the 
    target word indexed by word_index
  '''

  # YOUR CODE HERE
  result =[]
  nums = matrix.shape[0]
  target = matrix[target_word_index,:]
  word_ranking = {}
  for i in range(nums):
      if i!= target_word_index:
        word_simi = similarity_fn(matrix[i,:],target)
        word_ranking[i] = word_simi
  for k,v in sorted(word_ranking.items(),key=lambda item: item[1],reverse=True):
      result.append(k)
  return result


In [52]:
random_idx = random.randint(0, len(document_names)-1)
similarity_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]

In [42]:
  for sim_fn in similarity_fns:
    print('\nThe top most similar plays to "%s" using %s are:' % (document_names[random_idx], sim_fn.__qualname__))
    ranks = rank_plays(random_idx, td_matrix, sim_fn)
    for idx in range(0, 1):
      doc_id = ranks[idx]
      print('%d: %s' % (idx+1, document_names[doc_id]))

  word = 'juliet'
  vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
  for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on term-document frequency matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], td_matrix, sim_fn)
    for idx in range(0, 10):
      word_id = ranks[idx]
      print('%d: %s' % (idx+1, vocab[word_id]))

  word = 'juliet'
  vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
  for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on term-context frequency matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], tc_matrix, sim_fn)
    for idx in range(0, 10):
      word_id = ranks[idx]
      print('%d: %s' % (idx+1, vocab[word_id]))

  word = 'juliet'
  vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
  for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on tf-idf matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], tf_idf_matrix, sim_fn)
    for idx in range(0, 10):
      word_id = ranks[idx]
      print('%d: %s' % (idx+1, vocab[word_id]))

  word = 'juliet'
  vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
  for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on PPMI matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], PPMI_matrix, sim_fn)
    for idx in range(0, 10):
      word_id = ranks[idx]
      print('%d: %s' % (idx+1, vocab[word_id]))


The top most similar plays to "Henry V" using compute_cosine_similarity are:
1: King John

The top most similar plays to "Henry V" using compute_jaccard_similarity are:
1: Henry IV

The top most similar plays to "Henry V" using compute_dice_similarity are:
1: Henry IV

The 10 most similar words to "juliet" using compute_cosine_similarity on term-document frequency matrix are:
1: capulet
2: pump
3: laura
4: pitcher
5: behoveful
6: hurdle
7: capulets
8: petrucio
9: heartless
10: searchers

The 10 most similar words to "juliet" using compute_jaccard_similarity on term-document frequency matrix are:
1: tybalt
2: capulet
3: nurse
4: romeo
5: mercutio
6: friar
7: montague
8: laurence
9: paris
10: peter

The 10 most similar words to "juliet" using compute_dice_similarity on term-document frequency matrix are:
1: tybalt
2: capulet
3: nurse
4: romeo
5: mercutio
6: friar
7: montague
8: laurence
9: paris
10: peter

The 10 most similar words to "juliet" using compute_cosine_similarity on term-con

/Users/weihaoran/env/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


1: capulet
2: bedaub
3: howlings
4: demesnes
5: scaring
6: furred
7: umpire
8: hilding
9: nobleman
10: busied

The 10 most similar words to "juliet" using compute_jaccard_similarity on tf-idf matrix are:
1: benvolio
2: lucio
3: mercutio
4: capulet
5: tybalt
6: romeo
7: capulets
8: ghostly
9: montagues
10: procures

The 10 most similar words to "juliet" using compute_dice_similarity on tf-idf matrix are:
1: benvolio
2: lucio
3: mercutio
4: capulet
5: tybalt
6: romeo
7: capulets
8: ghostly
9: montagues
10: procures

The 10 most similar words to "juliet" using compute_cosine_similarity on PPMI matrix are:
1: nurse
2: tybalt
3: capulet
4: romeo
5: mistress
6: lady
7: silvia
8: provost
9: dead
10: waken

The 10 most similar words to "juliet" using compute_jaccard_similarity on PPMI matrix are:
1: silvia
2: nurse
3: tybalt
4: capulet
5: proteus
6: hamlet
7: leonato
8: romeo
9: orlando
10: cassio

The 10 most similar words to "juliet" using compute_dice_similarity on PPMI matrix are:
1: silvi

In [51]:
from sklearn.cluster import KMeans
import sklearn.cluster as cluster

## Categories Clustering

In [50]:
document_names

['Henry IV',
 'Alls well that ends well',
 'Loves Labours Lost',
 'Taming of the Shrew',
 'Antony and Cleopatra',
 'Coriolanus',
 'Hamlet',
 'A Midsummer nights dream',
 'Merry Wives of Windsor',
 'Romeo and Juliet',
 'Richard II',
 'King John',
 'macbeth',
 'Timon of Athens',
 'A Winters Tale',
 'The Tempest',
 'Henry VI Part 2',
 'As you like it',
 'Julius Caesar',
 'A Comedy of Errors',
 'Henry VIII',
 'Measure for measure',
 'Richard III',
 'Two Gentlemen of Verona',
 'Henry VI Part 1',
 'Much Ado about nothing',
 'Henry V',
 'Troilus and Cressida',
 'Twelfth Night',
 'Merchant of Venice',
 'Henry VI Part 3',
 'Othello',
 'Cymbeline',
 'King Lear',
 'Pericles',
 'Titus Andronicus']

In [55]:
Tragedies = {'Troilus and Cressida',
'Coriolanus',
'Titus Andronicus',
'Romeo and Juliet',
'Timon of Athens',
'Julius Caesar',
'macbeth',
'Hamlet',
'King Lear',
'Othello',
'Antony and Cleopatra',
'Cymbeline'}
Histories = {'King John',
'Richard II',
'Henry IV',
'Henry V',
'Henry VI Part 1',
'Henry VI Part 2',
'Henry VI Part 3',
'Richard III',
'Henry VIII',
'Edward III'}
Comedies = {'The Tempest',
'Two Gentlemen of Verona',
'Merry Wives of Windsor',
'Measure for measure',
'A Comedy of Errors',
'Much Ado about nothing',
'Loves Labours Lost',
'A Midsummer nights dream',
'Merchant of Venice',
'As you like it',
'Taming of the Shrew',
'Alls well that ends well',
'Twelfth Night',
'A Winters Tale', 
'Pericles'}

Flags notation: 0 stands for **Comedies**, 1 stands for **Histories** and 2 stands for **Tragedies**

In [56]:
flags = []
for i in range(len(document_names)):
    if document_names[i] in Tragedies:
        flags.append(2)
    if document_names[i] in Histories:
        flags.append(1)
    if document_names[i] in Comedies:
        flags.append(0)

#### k-means

In [80]:
k = 3
kmeans = KMeans(n_clusters=k, init='k-means++')
kmeans.fit_predict(td_matrix.T)
pred_km = kmeans.labels_

In [81]:
pred_km

array([0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 2, 2, 2, 2, 0, 2, 0, 1, 2, 2, 0, 1,
       0, 2, 2, 1, 0, 0, 1, 1, 0, 0, 0, 0, 2, 2], dtype=int32)

In [82]:
flags

[1,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 0,
 2]

In [85]:
for i in range(len(pred_km)):
    if pred_km[i] ==2:
        print("Comedies: "+document_names[i])
    elif pred_km[i] == 0:
        print("Tragedies:"+document_names[i])
    else :print("Histories:" + document_names[i])

Tragedies:Henry IV
Histories:Alls well that ends well
Histories:Loves Labours Lost
Histories:Taming of the Shrew
Tragedies:Antony and Cleopatra
Tragedies:Coriolanus
Tragedies:Hamlet
Comedies: A Midsummer nights dream
Histories:Merry Wives of Windsor
Histories:Romeo and Juliet
Comedies: Richard II
Comedies: King John
Comedies: macbeth
Comedies: Timon of Athens
Tragedies:A Winters Tale
Comedies: The Tempest
Tragedies:Henry VI Part 2
Histories:As you like it
Comedies: Julius Caesar
Comedies: A Comedy of Errors
Tragedies:Henry VIII
Histories:Measure for measure
Tragedies:Richard III
Comedies: Two Gentlemen of Verona
Comedies: Henry VI Part 1
Histories:Much Ado about nothing
Tragedies:Henry V
Tragedies:Troilus and Cressida
Histories:Twelfth Night
Histories:Merchant of Venice
Tragedies:Henry VI Part 3
Tragedies:Othello
Tragedies:Cymbeline
Tragedies:King Lear
Comedies: Pericles
Comedies: Titus Andronicus
